In [30]:
import pandas as pd

# Import Data / Transform into Dataframes

In [31]:
users_df = pd.read_csv('data/users.dat', delimiter='::', engine='python', names=['uid', 'gender', 'age', 'occupation', 'zip'])
movies_df = pd.read_csv('data/movies.dat', delimiter='::', engine='python', names=['movie_id', 'title', 'genre'], encoding='latin-1')
ratings_df = pd.read_csv('data/ratings.dat', delimiter='::', engine='python', names=['uid', 'movie_id', 'rating', 'ts'])

In [32]:
users_df.head()

,uid,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [33]:
movies_df.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [34]:
ratings_df.head()

,uid,movie_id,rating,ts
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


# Join Dataframes

In [35]:
user_ratings_df = pd.merge(ratings_df, users_df, on='uid')
df = pd.merge(user_ratings_df, movies_df, on='movie_id')
df.head()

,uid,movie_id,rating,ts,gender,age,occupation,zip,title,genre
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,F,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,F,1,10,48067,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,F,1,10,48067,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,F,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy


# Process Dataset

In [36]:
df.sort_values(by='ts', inplace=True)
df

,uid,movie_id,rating,ts,gender,age,occupation,zip,title,genre
1000138,6040,858,4,956703932,M,25,6,11106,"Godfather, The (1972)",Action|Crime|Drama
1000153,6040,2384,4,956703954,M,25,6,11106,Babe: Pig in the City (1998),Children's|Comedy
999873,6040,593,5,956703954,M,25,6,11106,"Silence of the Lambs, The (1991)",Drama|Thriller
1000007,6040,1961,4,956703977,M,25,6,11106,Rain Man (1988),Drama
1000192,6040,2019,5,956703977,M,25,6,11106,Seven Samurai (The Magnificent Seven) (Shichin...,Action|Drama
...,...,...,...,...,...,...,...,...,...,...
825793,4958,2399,1,1046454338,M,18,7,55403,Santa Claus: The Movie (1985),Adventure|Children's|Fantasy
825438,4958,1407,5,1046454443,M,18,7,55403,Scream (1996),Horror|Thriller
825724,4958,3264,4,1046454548,M,18,7,55403,Buffy the Vampire Slayer (1992),Comedy|Horror
825731,4958,2634,3,1046454548,M,18,7,55403,"Mummy, The (1959)",Horror


## Bin Age Groups + One Hot Encoding for age_group and gender

In [37]:
df = df.reset_index(drop=True)
bins = [0, 12, 19, 60, float('inf')]
labels = ['child', 'teen', 'adult', 'senior']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)
df = pd.get_dummies(df, columns=['age_group', 'gender'], dtype=int)
mapping = {
    1: 1,
    18: 2,
    25: 3,
    35: 4,
    45: 5,
    50: 6,
    56: 7
}

df['age_sparse'] = df['age'].map(mapping)

## One Hot Encoding for Genres

In [38]:
df['genres_list'] = df['genre'].str.split('|')
df_exploded = df.explode('genres_list')
one_hot = pd.get_dummies(df_exploded['genres_list'], dtype=int)
one_hot_exploded = one_hot.groupby(df_exploded.index).max()
df = pd.concat([df, one_hot_exploded], axis=1)

In [39]:
print(df.iloc[0])

uid                                   6040
movie_id                               858
rating                                   4
ts                               956703932
age                                     25
occupation                               6
zip                                  11106
title                Godfather, The (1972)
genre                   Action|Crime|Drama
age_group_child                          0
age_group_teen                           0
age_group_adult                          1
age_group_senior                         0
gender_F                                 0
gender_M                                 1
age_sparse                               3
genres_list         [Action, Crime, Drama]
Action                                   1
Adventure                                0
Animation                                0
Children's                               0
Comedy                                   0
Crime                                    1
Documentary

In [40]:
df.drop(columns=['age', 'genres_list', 'genre', 'title', 'ts'], inplace=True, axis=1)
df = df[df['rating'] != 3]

## Bin ratings

In [41]:
print(df.columns)

Index(['uid', 'movie_id', 'rating', 'occupation', 'zip', 'age_group_child',
       'age_group_teen', 'age_group_adult', 'age_group_senior', 'gender_F',
       'gender_M', 'age_sparse', 'Action', 'Adventure', 'Animation',
       'Children's', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
       'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
       'Thriller', 'War', 'Western'],
      dtype='object')


In [42]:
df['rating'].value_counts()

rating
4    348971
5    226310
2    107557
1     56174
Name: count, dtype: int64

In [43]:
df['rating'] = df['rating'].mask(df['rating'] < 3, 0)
df['rating'] = df['rating'].mask(df['rating'] > 3, 1)
df

,uid,movie_id,rating,occupation,zip,age_group_child,age_group_teen,age_group_adult,age_group_senior,gender_F,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,6040,858,1,6,11106,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6040,2384,1,6,11106,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6040,593,1,6,11106,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,6040,1961,1,6,11106,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6040,2019,1,6,11106,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000203,4958,3489,1,7,55403,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1000204,4958,2399,0,7,55403,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1000205,4958,1407,1,7,55403,0,1,0,0,0,...,0,0,1,0,0,0,0,1,0,0
1000206,4958,3264,1,7,55403,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [44]:
df['rating'].value_counts()

rating
1    575281
0    163731
Name: count, dtype: int64

In [45]:
df.to_csv('data/dataset.csv')